In [1]:
import pandas as pd

prof_df = pd.read_csv('prof_dat.csv')
comments_df = pd.read_csv('all_comments.csv')

prof_df.rename(columns={'Quality Rating': 'Prof Quality Rating'}, inplace=True)
comments_df.drop(columns=['Link', 'Unnamed: 0'], inplace=True)
comments_df.rename(columns={'Quality Rating': 'Comment Quality Rating'}, inplace=True)

In [47]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
import re
from tqdm import tqdm

registrar_df = pd.read_csv('registrar_data.csv')

course_df = pd.read_csv('ucdavis_courses.csv')
course_df['Subject Name'] = [re.sub(r'\(\w+\)', '', sub_name) for sub_name in course_df['Subject Name']]

registrar_course_df = pd.merge(registrar_df, course_df, left_on='CourseId', right_on='Course Code', how='left')

prerequisites_df = pd.DataFrame(columns=['Course Code', 'Prerequisite Code'])
c=0


for index, rows in tqdm(registrar_course_df.iterrows()):
    registrar_prof_name = rows['ProfName']
    match_scores_name = [(fuzzywuzzy.fuzz.token_set_ratio(registrar_prof_name, profName), profName) for profName in set(prof_df['Name'])]
    match_scores_dep = [(fuzzywuzzy.fuzz.token_set_ratio(rows['Subject Name'], depName), depName) for depName in set(prof_df['Department'])]
    match_scores_name.sort(reverse=True)
    match_scores_dep.sort(reverse=True)
    
    if match_scores_name[0][0] < 70:
        if match_scores_name[0][0] + match_scores_dep[0][0] <= 140 or match_scores_name[0][0] < 65:
            # print(registrar_prof_name, ':', match_scores_name[0][1], match_scores_name[0][0])
            # print(rows['Subject Name'], ':', match_scores_dep[0][1], match_scores_dep[0][0])
            # print()
            continue

    registrar_course_df.at[index, 'ProfName'] = match_scores_name[0][1]
    registrar_course_df.at[index, 'Subject Name'] = match_scores_dep[0][1]
    course_units = rows['Course Units'].strip()
    course_units = re.sub(r' unit[s]*', '', course_units)
    course_units = re.sub(r'\d+-', '', course_units)
    registrar_course_df.at[index, 'Course Units'] = course_units
    prerequisites = re.findall(r'(\w+ \d+[A-Z]*)', str(rows['Prerequisites']))
    for pr in prerequisites:
        prerequisites_df.loc[len(prerequisites_df)] = [rows['Course Code'], pr]
    c+=1

print('Number of matches - ', c)

4576it [03:47, 20.12it/s]

Number of matches -  4126


In [48]:
registrar_course_df['Course Description'] = [first + ': ' + last 
                                             for first, last in zip(list(registrar_course_df['Course Name']), list(registrar_course_df['Course Description']))]
registrar_course_df['Course Units'] = [units.replace(' units', '') for units in registrar_course_df['Course Units']]
registrar_course_df.drop(columns=['Prerequisites', 'CourseId', 'Course Name', 'Subject Code'], inplace=True)
registrar_course_df = registrar_course_df.convert_dtypes()
registrar_course_df

,ProfName,Quarter,CRN,Subject Name,Course Code,Course Description,Course Units
0,Moradewun Adejunmobi,202401,44239,American Studies,AAS 017,Women in African Societies: Gender relations i...,4
1,Bettina Ng'Weno,202401,10001,American Studies,AAS 018,Introduction to Caribbean Studies: Introductio...,4
2,Elisa White,202401,10003,American Studies,AAS 050,Black Popular Culture: Survey of the African A...,4
3,Elizabeth Mukiibi,202401,44240,American Studies,AAS 101,Introduction to Research in the Afro-American ...,4
4,Elisa White,202401,10011,American Studies,AAS 107B,African Descent Communities & Culture in North...,4
...,...,...,...,...,...,...,...
4571,Joseph Mitchell,202410,43203,Physics,PHY 001A,Principles of Physics: Mechanics. Introduction...,3
4572,Brie Tripp,202410,40252,Neurobiology & Behavior,NPB 143,Neurobiology & Pathophysiology of Mental Illne...,3
4573,Armela Keqi,202401,36967,Physics,PHY 007B,General Physics: Continuation of PHY 007A.,4
4574,Andrew McElrone,202410,49118,Viticulture & Enology,VEN 290,Seminar: Seminar.,1


In [50]:
print(len(set(zip(list(registrar_course_df['CRN']), list(registrar_course_df['Quarter'])))))

4568


In [49]:
import sqlite3

conn = sqlite3.connect('smartAggie.sqlite')
# course_info
registrar_course_df.to_sql(name='Courses', index=False, con=conn)
# prerequisites
# prerequisites_df.to_sql(name='Prerequisites', index=False, con=conn)
# prof_df
# prof_df.to_sql(name='Professors', index=False, con=conn)
# comments_df
# comments_df.to_sql(name='Comments', index=False, con=conn)

4576